# Inner join - intuition

Dans la partie précédente, on a découvert les cross joins.

On a démarré par les cross joins parce que ça permet de bâtir de l'intuition sur les autres formes de joins.

Dans ce notebook, on va découvrir que tous les autres joins peuvent être imaginés comme des cross_joins avec un filtre

In [1]:
import pandas as pd
import duckdb
import io

## Cross join avec plusieurs colonnes dans la table

Imaginons qu'on ait deux tables, salaries & seniorities:

In [3]:
csv = '''
salary,employee_id
2000,1
2500,2
2200,3
'''

csv2 = '''
employee_id,seniority
1,2ans
2,4ans
'''

salaries = pd.read_csv(io.StringIO(csv))
senorities = pd.read_csv(io.StringIO(csv2))

display(salaries)
display(senorities)

,salary,employee_id
0,2000,1
1,2500,2
2,2200,3


,employee_id,seniority
0,1,2ans
1,2,4ans


In [4]:
cross_join_query = """
SELECT
    *
FROM salaries
CROSS JOIN senorities
"""

duckdb.sql(cross_join_query)

┌────────┬─────────────┬─────────────┬───────────┐
│ salary │ employee_id │ employee_id │ seniority │
│ int64  │    int64    │    int64    │  varchar  │
├────────┼─────────────┼─────────────┼───────────┤
│   2000 │           1 │           1 │ 2ans      │
│   2500 │           2 │           1 │ 2ans      │
│   2200 │           3 │           1 │ 2ans      │
│   2000 │           1 │           2 │ 4ans      │
│   2500 │           2 │           2 │ 4ans      │
│   2200 │           3 │           2 │ 4ans      │
└────────┴─────────────┴─────────────┴───────────┘

## Cross join avec un filtre = inner join !

Si on décide maintenant d'appliquer un filtre avec la clause WHERE, on peut se limiter aux entrées pour lesquelles les deux colonnes employee_id sont égales:

In [5]:
cross_join_query_where = """
SELECT
    *
FROM salaries
CROSS JOIN senorities
WHERE salaries.employee_id = senorities.employee_id
"""

duckdb.sql(cross_join_query_where)

┌────────┬─────────────┬─────────────┬───────────┐
│ salary │ employee_id │ employee_id │ seniority │
│ int64  │    int64    │    int64    │  varchar  │
├────────┼─────────────┼─────────────┼───────────┤
│   2000 │           1 │           1 │ 2ans      │
│   2500 │           2 │           2 │ 4ans      │
└────────┴─────────────┴─────────────┴───────────┘

## Plus clean:

In [8]:
cross_join_query_where2 = """
SELECT salary, salaries.employee_id, seniority 
FROM salaries
CROSS JOIN senorities
WHERE salaries.employee_id = senorities.employee_id
"""
duckdb.sql(cross_join_query_where2)

┌────────┬─────────────┬───────────┐
│ salary │ employee_id │ seniority │
│ int64  │    int64    │  varchar  │
├────────┼─────────────┼───────────┤
│   2000 │           1 │ 2ans      │
│   2500 │           2 │ 4ans      │
└────────┴─────────────┴───────────┘

L'utilisation de la syntaxe de l'INNER JOIN nous donne exactement le même résultat que le CROSS JOIN avec un FILTRE:

In [10]:
cross_join_query_inner = """
SELECT
    salary, salaries.employee_id, seniority
FROM salaries
INNER JOIN senorities
ON salaries.employee_id = senorities.employee_id
"""

duckdb.sql(cross_join_query_inner)

┌────────┬─────────────┬───────────┐
│ salary │ employee_id │ seniority │
│ int64  │    int64    │  varchar  │
├────────┼─────────────┼───────────┤
│   2000 │           1 │ 2ans      │
│   2500 │           2 │ 4ans      │
└────────┴─────────────┴───────────┘

⇒ En définitive, toutes les sortes de joins peuvent être ramenées à un produit cartésien auquel on applique un filtre.


Dans le prochain notebook, on va voir quelques exemples d'INNER JOIN

# Inner join - exemples

Dans le notebook précédent, on a bâti autour de l'intuition qu'un inner join peut-être conceptualisé comme un cross join avec un where.

Maintenant, on va passer à la pratique pour progresser sur la syntaxe

In [11]:
import pandas as pd
import duckdb
import io

## Data

In [12]:
# Table des commandes:
orders_data = {
    'order_id': [1, 2, 3, 4, 5],
    'customer_id': [101, 102, 103, 104, 105]
}

df_orders = pd.DataFrame(orders_data)
df_orders

,order_id,customer_id
0,1,101
1,2,102
2,3,103
3,4,104
4,5,105


In [13]:
# Table des clients
customers_data = {
    'customer_id': [101, 102, 103, 104, 105, 106],
    'customer_name': ["Toufik", "Daniel", "Tancrède", "Kaouter", "Jean-Nicolas", "David"]
}

df_customers = pd.DataFrame(customers_data)
df_customers

,customer_id,customer_name
0,101,Toufik
1,102,Daniel
2,103,Tancrède
3,104,Kaouter
4,105,Jean-Nicolas
5,106,David


In [14]:
# Table des produits
p_names = ["Laptop", "Ipad", "Livre", "Petitos"]
products_data = {
    'product_id': [101, 103, 104, 105],
    'product_name': p_names,
    'product_price': [800, 400, 30, 2]
}

df_products = pd.DataFrame(products_data)
df_products

,product_id,product_name,product_price
0,101,Laptop,800
1,103,Ipad,400
2,104,Livre,30
3,105,Petitos,2


In [15]:
# Détail des commandes
order_details_data = {
    'order_id': [1, 2, 3, 4, 5],
    'product_id': [102, 104, 101, 103, 105],
    'quantity': [2, 1, 3, 2, 1]
}

df_order_details = pd.DataFrame(order_details_data)
df_order_details

,order_id,product_id,quantity
0,1,102,2
1,2,104,1
2,3,101,3
3,4,103,2
4,5,105,1


## Inner joins pour rassembler les tables

### Exercice 1: inner join pour rassembler les commandes avec les détails

In [25]:
detailed_order = pd.merge(
    df_orders,
    df_order_details,
    on = "order_id",
    how = "inner"
)

detailed_order

,order_id,customer_id,product_id,quantity
0,1,101,102,2
1,2,102,104,1
2,3,103,101,3
3,4,104,103,2
4,5,105,105,1


In [28]:
query = """
SELECT 
    df_orders.order_id, customer_id, product_id, quantity
FROM df_orders
INNER JOIN df_order_details
ON df_orders.order_id = df_order_details.order_id
"""

duckdb.sql(query)

┌──────────┬─────────────┬────────────┬──────────┐
│ order_id │ customer_id │ product_id │ quantity │
│  int64   │    int64    │   int64    │  int64   │
├──────────┼─────────────┼────────────┼──────────┤
│        1 │         101 │        102 │        2 │
│        2 │         102 │        104 │        1 │
│        3 │         103 │        101 │        3 │
│        4 │         104 │        103 │        2 │
│        5 │         105 │        105 │        1 │
└──────────┴─────────────┴────────────┴──────────┘

In [31]:
query2 = """
SELECT 
    *
FROM df_orders
INNER JOIN df_order_details
USING (order_id)
"""
duckdb.sql(query2)

┌──────────┬─────────────┬────────────┬──────────┐
│ order_id │ customer_id │ product_id │ quantity │
│  int64   │    int64    │   int64    │  int64   │
├──────────┼─────────────┼────────────┼──────────┤
│        1 │         101 │        102 │        2 │
│        2 │         102 │        104 │        1 │
│        3 │         103 │        101 │        3 │
│        4 │         104 │        103 │        2 │
│        5 │         105 │        105 │        1 │
└──────────┴─────────────┴────────────┴──────────┘

### Exercice 2:  inner join pour rassembler les commandes détaillées avec les clients

In [35]:
order_customer = pd.merge(
    detailed_order,
    df_customers,
    on = 'customer_id',
    how = 'inner'
)
order_customer

,order_id,customer_id,product_id,quantity,customer_name
0,1,101,102,2,Toufik
1,2,102,104,1,Daniel
2,3,103,101,3,Tancrède
3,4,104,103,2,Kaouter
4,5,105,105,1,Jean-Nicolas


In [36]:
order_customer_query = """
SELECT
    *
FROM detailed_order
INNER JOIN df_customers
USING (customer_id)
"""
duckdb.sql(order_customer_query)

┌──────────┬─────────────┬────────────┬──────────┬───────────────┐
│ order_id │ customer_id │ product_id │ quantity │ customer_name │
│  int64   │    int64    │   int64    │  int64   │    varchar    │
├──────────┼─────────────┼────────────┼──────────┼───────────────┤
│        1 │         101 │        102 │        2 │ Toufik        │
│        2 │         102 │        104 │        1 │ Daniel        │
│        3 │         103 │        101 │        3 │ Tancrède      │
│        4 │         104 │        103 │        2 │ Kaouter       │
│        5 │         105 │        105 │        1 │ Jean-Nicolas  │
└──────────┴─────────────┴────────────┴──────────┴───────────────┘

### Exercice 3: inner join pour rassembler les commandes client détaillées avec les produits

In [39]:
order_customer_products = pd.merge(
    order_customer,
    df_products,
    on = 'product_id',
    how = 'inner'
)
order_customer_products

,order_id,customer_id,product_id,quantity,customer_name,product_name,product_price
0,2,102,104,1,Daniel,Livre,30
1,3,103,101,3,Tancrède,Laptop,800
2,4,104,103,2,Kaouter,Ipad,400
3,5,105,105,1,Jean-Nicolas,Petitos,2


In [40]:
order_customer_products_query = """
SELECT
    *
FROM order_customer
INNER JOIN df_products
USING (product_id)
"""
duckdb.sql(order_customer_products_query)

┌──────────┬─────────────┬────────────┬──────────┬───────────────┬──────────────┬───────────────┐
│ order_id │ customer_id │ product_id │ quantity │ customer_name │ product_name │ product_price │
│  int64   │    int64    │   int64    │  int64   │    varchar    │   varchar    │     int64     │
├──────────┼─────────────┼────────────┼──────────┼───────────────┼──────────────┼───────────────┤
│        2 │         102 │        104 │        1 │ Daniel        │ Livre        │            30 │
│        3 │         103 │        101 │        3 │ Tancrède      │ Laptop       │           800 │
│        4 │         104 │        103 │        2 │ Kaouter       │ Ipad         │           400 │
│        5 │         105 │        105 │        1 │ Jean-Nicolas  │ Petitos      │             2 │
└──────────┴─────────────┴────────────┴──────────┴───────────────┴──────────────┴───────────────┘

In [41]:
import random
import pandas as pd
import duckdb
from datetime import datetime, timedelta

# Création de la donnée

In [42]:
univers = ["Électronique", "Mode", "Maison"]

categories_par_univers = {
    "Électronique": ["Téléphones", "Ordinateurs"],
    "Mode": ["Vêtements", "Accessoires"],
    "Maison": ["Meubles", "Décoration"]
}

noms_produits = {
    "Téléphones": ["iPhone 13", "Samsung Galaxy S21", "Google Pixel 6", "OnePlus 9", "Xiaomi Mi 11", "Sony Xperia 5 III", "Huawei P40 Pro", "LG Velvet", "Motorola Edge", "Nokia 8.3"],
    "Ordinateurs": ["MacBook Pro", "Dell XPS 15", "HP Spectre x360", "Lenovo ThinkPad", "Asus ROG Zephyrus", "Microsoft Surface Laptop", "Acer Predator Helios", "Razer Blade", "MSI Prestige", "LG Gram"],
    "Vêtements": ["Chemise en lin", "Robe d'été", "Jeans slim", "Veste en cuir", "Pull en laine", "Pantalon chino", "T-shirt graphique", "Blouse à volants", "Blazer ajusté", "Short en denim"],
    "Accessoires": ["Montre élégante", "Sac à dos moderne", "Lunettes de soleil", "Ceinture en cuir", "Écharpe en soie", "Boucles d'oreilles", "Chapeau en feutre", "Bracelet en métal", "Cravate en soie", "Portefeuille en cuir"],
    "Meubles": ["Canapé modulaire", "Table à manger en bois", "Lit king-size", "Chaise ergonomique", "Bureau en verre", "Étagère murale", "Buffet en bois", "Fauteuil inclinable", "Table basse moderne", "Commode à tiroirs"],
    "Décoration": ["Vase en céramique", "Tableau abstrait", "Bougie parfumée", "Coussins décoratifs", "Horloge murale", "Plante d'intérieur", "Suspension lumineuse", "Miroir encadré", "Tapis tissé", "Statuette en bronze"]
}

donnees = []
produit_id_counter = 1
categorie_id = 0

for univers_id, univers_name in enumerate(univers):
    for categorie in categories_par_univers[univers_name]:
        categorie_id += 1
        for _ in range(15):  # Créer 15 exemples de produits par catégorie
            produit = {
                "produit_id": produit_id_counter,
                "univers_id": univers_id,
                "univers_name": univers_name,
                "categorie_name": categorie,
                "categorie_id": categorie_id,
                "nom": random.choice(noms_produits[categorie]),
                "prix_unitaire": round(random.uniform(10, 1000), 2)
            }
            donnees.append(produit)
            produit_id_counter += 1

df = pd.DataFrame(donnees)
products = df[["produit_id", "prix_unitaire", "nom"]]
categorie_produit = df[["categorie_id", "categorie_name", "produit_id"]]
univers_categorie = df[["univers_id", "univers_name", "categorie_id"]].drop_duplicates()


# Un mois de ventes:

date_debut = datetime(2023, 7, 1)
date_fin = datetime(2023, 7, 31)
jours_dans_le_mois = (date_fin - date_debut).days + 1

ventes = []

for jour in range(jours_dans_le_mois):
    date_vente = date_debut + timedelta(days=jour)
    n_sales_that_day = range(1, random.randint(10, 300))
    for vente in n_sales_that_day:
        products_in_that_sale = products.sample(random.randint(1, 36))
        for _, row in products_in_that_sale.iterrows():
            quantite_vendue = random.randint(1, 10)
            montant_total = row["prix_unitaire"] * quantite_vendue
            ventes.append({
                "date": date_vente,
                "produit_id": row["produit_id"],
                "quantite_vendue": quantite_vendue,
                "prix_unitaire": row["prix_unitaire"],
                "montant_total": montant_total
            })
del(df)
# Créer une DataFrame Pandas pour les ventes
ventes_df = pd.DataFrame(ventes)
ventes_df.shape

(87821, 5)

# Présentation de la donnée

Vous avez à votre disposition une table de ventes:

Une table produits:

In [43]:
ventes_df

,date,produit_id,quantite_vendue,prix_unitaire,montant_total
0,2023-07-01,21,2,705.91,1411.82
1,2023-07-01,20,10,82.69,826.90
2,2023-07-01,33,10,859.70,8597.00
3,2023-07-01,38,3,679.42,2038.26
4,2023-07-01,28,3,345.97,1037.91
...,...,...,...,...,...
87816,2023-07-31,32,1,730.35,730.35
87817,2023-07-31,8,8,987.62,7900.96
87818,2023-07-31,50,4,307.03,1228.12
87819,2023-07-31,63,2,298.01,596.02


In [44]:
products

,produit_id,prix_unitaire,nom
0,1,248.45,Sony Xperia 5 III
1,2,604.37,iPhone 13
2,3,126.29,iPhone 13
3,4,632.65,OnePlus 9
4,5,582.97,OnePlus 9
...,...,...,...
85,86,959.45,Bougie parfumée
86,87,843.47,Tableau abstrait
87,88,690.68,Vase en céramique
88,89,332.52,Coussins décoratifs


---

Une table avec les catégories:

Une table avec les univers

In [45]:
categorie_produit

,categorie_id,categorie_name,produit_id
0,1,Téléphones,1
1,1,Téléphones,2
2,1,Téléphones,3
3,1,Téléphones,4
4,1,Téléphones,5
...,...,...,...
85,6,Décoration,86
86,6,Décoration,87
87,6,Décoration,88
88,6,Décoration,89


In [46]:
univers_categorie

,univers_id,univers_name,categorie_id
0,0,Électronique,1
15,0,Électronique,2
30,1,Mode,3
45,1,Mode,4
60,2,Maison,5
75,2,Maison,6


## Exercice: savoir à quel univers correspond chaque vente
(plus tard, cette jointure servira à savoir quels sont les univers qui rapportent le plus...)

In [52]:
vp = pd.merge(
    ventes_df,
    products[['produit_id', 'nom']],
    on = 'produit_id',
    how = 'inner'
)
vp

,date,produit_id,quantite_vendue,prix_unitaire,montant_total,nom
0,2023-07-01,21,2,705.91,1411.82,Acer Predator Helios
1,2023-07-01,21,10,705.91,7059.10,Acer Predator Helios
2,2023-07-01,21,6,705.91,4235.46,Acer Predator Helios
3,2023-07-01,21,6,705.91,4235.46,Acer Predator Helios
4,2023-07-01,21,4,705.91,2823.64,Acer Predator Helios
...,...,...,...,...,...,...
87816,2023-07-31,63,5,298.01,1490.05,Commode à tiroirs
87817,2023-07-31,63,3,298.01,894.03,Commode à tiroirs
87818,2023-07-31,63,1,298.01,298.01,Commode à tiroirs
87819,2023-07-31,63,3,298.01,894.03,Commode à tiroirs


In [55]:
vp_query = """
SELECT date, ventes_df.produit_id, quantite_vendue, ventes_df.prix_unitaire, montant_total, nom
FROM ventes_df
INNER JOIN products
USING(produit_id)
"""
duckdb.sql(vp_query).df()

,date,produit_id,quantite_vendue,prix_unitaire,montant_total,nom
0,2023-07-01,21,2,705.91,1411.82,Acer Predator Helios
1,2023-07-01,20,10,82.69,826.90,Lenovo ThinkPad
2,2023-07-01,33,10,859.70,8597.00,Jeans slim
3,2023-07-01,38,3,679.42,2038.26,Chemise en lin
4,2023-07-01,28,3,345.97,1037.91,Dell XPS 15
...,...,...,...,...,...,...
87816,2023-07-30,50,3,307.03,921.09,Sac à dos moderne
87817,2023-07-30,44,2,45.84,91.68,Blouse à volants
87818,2023-07-30,50,5,307.03,1535.15,Sac à dos moderne
87819,2023-07-30,79,9,147.78,1330.02,Suspension lumineuse


In [58]:
vp_cat = pd.merge(
    vp,
    categorie_produit,
    on = 'produit_id',
    how = 'inner'
)
vp_cat

,date,produit_id,quantite_vendue,prix_unitaire,montant_total,nom,categorie_id,categorie_name
0,2023-07-01,21,2,705.91,1411.82,Acer Predator Helios,2,Ordinateurs
1,2023-07-01,21,10,705.91,7059.10,Acer Predator Helios,2,Ordinateurs
2,2023-07-01,21,6,705.91,4235.46,Acer Predator Helios,2,Ordinateurs
3,2023-07-01,21,6,705.91,4235.46,Acer Predator Helios,2,Ordinateurs
4,2023-07-01,21,4,705.91,2823.64,Acer Predator Helios,2,Ordinateurs
...,...,...,...,...,...,...,...,...
87816,2023-07-31,63,5,298.01,1490.05,Commode à tiroirs,5,Meubles
87817,2023-07-31,63,3,298.01,894.03,Commode à tiroirs,5,Meubles
87818,2023-07-31,63,1,298.01,298.01,Commode à tiroirs,5,Meubles
87819,2023-07-31,63,3,298.01,894.03,Commode à tiroirs,5,Meubles


In [60]:
vp_cat_query = """
SELECT
    *
FROM vp
INNER JOIN categorie_produit
USING(produit_id)
"""
duckdb.sql(vp_cat_query).df()

,date,produit_id,quantite_vendue,prix_unitaire,montant_total,nom,categorie_id,categorie_name
0,2023-07-01,21,2,705.91,1411.82,Acer Predator Helios,2,Ordinateurs
1,2023-07-01,21,10,705.91,7059.10,Acer Predator Helios,2,Ordinateurs
2,2023-07-01,21,6,705.91,4235.46,Acer Predator Helios,2,Ordinateurs
3,2023-07-01,21,6,705.91,4235.46,Acer Predator Helios,2,Ordinateurs
4,2023-07-01,21,4,705.91,2823.64,Acer Predator Helios,2,Ordinateurs
...,...,...,...,...,...,...,...,...
87816,2023-07-31,63,5,298.01,1490.05,Commode à tiroirs,5,Meubles
87817,2023-07-31,63,3,298.01,894.03,Commode à tiroirs,5,Meubles
87818,2023-07-31,63,1,298.01,298.01,Commode à tiroirs,5,Meubles
87819,2023-07-31,63,3,298.01,894.03,Commode à tiroirs,5,Meubles


In [63]:
vp_cat_univers = pd.merge(
    vp_cat,
    univers_categorie,
    on = 'categorie_id',
    how = 'inner'
)
vp_cat_univers

,date,produit_id,quantite_vendue,prix_unitaire,montant_total,nom,categorie_id,categorie_name,univers_id,univers_name
0,2023-07-01,21,2,705.91,1411.82,Acer Predator Helios,2,Ordinateurs,0,Électronique
1,2023-07-01,21,10,705.91,7059.10,Acer Predator Helios,2,Ordinateurs,0,Électronique
2,2023-07-01,21,6,705.91,4235.46,Acer Predator Helios,2,Ordinateurs,0,Électronique
3,2023-07-01,21,6,705.91,4235.46,Acer Predator Helios,2,Ordinateurs,0,Électronique
4,2023-07-01,21,4,705.91,2823.64,Acer Predator Helios,2,Ordinateurs,0,Électronique
...,...,...,...,...,...,...,...,...,...,...
87816,2023-07-31,63,5,298.01,1490.05,Commode à tiroirs,5,Meubles,2,Maison
87817,2023-07-31,63,3,298.01,894.03,Commode à tiroirs,5,Meubles,2,Maison
87818,2023-07-31,63,1,298.01,298.01,Commode à tiroirs,5,Meubles,2,Maison
87819,2023-07-31,63,3,298.01,894.03,Commode à tiroirs,5,Meubles,2,Maison


In [65]:
vp_cat_univers_query = """
SELECT
    *
FROM vp_cat
INNER JOIN univers_categorie
USING (categorie_id)
"""
duckdb.sql(vp_cat_univers_query).df()

,date,produit_id,quantite_vendue,prix_unitaire,montant_total,nom,categorie_id,categorie_name,univers_id,univers_name
0,2023-07-01,21,2,705.91,1411.82,Acer Predator Helios,2,Ordinateurs,0,Électronique
1,2023-07-01,21,10,705.91,7059.10,Acer Predator Helios,2,Ordinateurs,0,Électronique
2,2023-07-01,21,6,705.91,4235.46,Acer Predator Helios,2,Ordinateurs,0,Électronique
3,2023-07-01,21,6,705.91,4235.46,Acer Predator Helios,2,Ordinateurs,0,Électronique
4,2023-07-01,21,4,705.91,2823.64,Acer Predator Helios,2,Ordinateurs,0,Électronique
...,...,...,...,...,...,...,...,...,...,...
87816,2023-07-31,63,5,298.01,1490.05,Commode à tiroirs,5,Meubles,2,Maison
87817,2023-07-31,63,3,298.01,894.03,Commode à tiroirs,5,Meubles,2,Maison
87818,2023-07-31,63,1,298.01,298.01,Commode à tiroirs,5,Meubles,2,Maison
87819,2023-07-31,63,3,298.01,894.03,Commode à tiroirs,5,Meubles,2,Maison


## en une fois

In [67]:
vp_query2 = """
SELECT *
FROM ventes_df
INNER JOIN categorie_produit
USING(produit_id)
INNER JOIN univers_categorie
USING (categorie_id)
"""
duckdb.sql(vp_query2).df()

,date,produit_id,quantite_vendue,prix_unitaire,montant_total,categorie_id,categorie_name,univers_id,univers_name
0,2023-07-01,21,2,705.91,1411.82,2,Ordinateurs,0,Électronique
1,2023-07-01,20,10,82.69,826.90,2,Ordinateurs,0,Électronique
2,2023-07-01,33,10,859.70,8597.00,3,Vêtements,1,Mode
3,2023-07-01,38,3,679.42,2038.26,3,Vêtements,1,Mode
4,2023-07-01,28,3,345.97,1037.91,2,Ordinateurs,0,Électronique
...,...,...,...,...,...,...,...,...,...
87816,2023-07-30,50,3,307.03,921.09,4,Accessoires,1,Mode
87817,2023-07-30,44,2,45.84,91.68,3,Vêtements,1,Mode
87818,2023-07-30,50,5,307.03,1535.15,4,Accessoires,1,Mode
87819,2023-07-30,79,9,147.78,1330.02,6,Décoration,2,Maison


In [71]:
vp2 = pd.merge(
    ventes_df,
    categorie_produit,
    on = 'produit_id',
    how = 'inner'
).merge(
    univers_categorie,
    on = 'categorie_id',
    how = 'inner'
)

vp2

,date,produit_id,quantite_vendue,prix_unitaire,montant_total,categorie_id,categorie_name,univers_id,univers_name
0,2023-07-01,21,2,705.91,1411.82,2,Ordinateurs,0,Électronique
1,2023-07-01,21,10,705.91,7059.10,2,Ordinateurs,0,Électronique
2,2023-07-01,21,6,705.91,4235.46,2,Ordinateurs,0,Électronique
3,2023-07-01,21,6,705.91,4235.46,2,Ordinateurs,0,Électronique
4,2023-07-01,21,4,705.91,2823.64,2,Ordinateurs,0,Électronique
...,...,...,...,...,...,...,...,...,...
87816,2023-07-31,63,5,298.01,1490.05,5,Meubles,2,Maison
87817,2023-07-31,63,3,298.01,894.03,5,Meubles,2,Maison
87818,2023-07-31,63,1,298.01,298.01,5,Meubles,2,Maison
87819,2023-07-31,63,3,298.01,894.03,5,Meubles,2,Maison


In [74]:
vp3 = ventes_df.merge(categorie_produit, on = 'produit_id', how = 'inner') \
        .merge(univers_categorie, on='categorie_id', how='inner')
vp3

,date,produit_id,quantite_vendue,prix_unitaire,montant_total,categorie_id,categorie_name,univers_id,univers_name
0,2023-07-01,21,2,705.91,1411.82,2,Ordinateurs,0,Électronique
1,2023-07-01,21,10,705.91,7059.10,2,Ordinateurs,0,Électronique
2,2023-07-01,21,6,705.91,4235.46,2,Ordinateurs,0,Électronique
3,2023-07-01,21,6,705.91,4235.46,2,Ordinateurs,0,Électronique
4,2023-07-01,21,4,705.91,2823.64,2,Ordinateurs,0,Électronique
...,...,...,...,...,...,...,...,...,...
87816,2023-07-31,63,5,298.01,1490.05,5,Meubles,2,Maison
87817,2023-07-31,63,3,298.01,894.03,5,Meubles,2,Maison
87818,2023-07-31,63,1,298.01,298.01,5,Meubles,2,Maison
87819,2023-07-31,63,3,298.01,894.03,5,Meubles,2,Maison
